<a href="https://colab.research.google.com/github/PIYALI-bhunia/Multimodal_Results/blob/main/MobileNetV2_Storing_Result_in_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import pathlib
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
data_root = pathlib.Path('/content/drive/MyDrive/Tobacco3482-jpg/')

print(data_root)
for item in data_root.iterdir():
  print(item)

/content/drive/MyDrive/Tobacco3482-jpg
/content/drive/MyDrive/Tobacco3482-jpg/Letter
/content/drive/MyDrive/Tobacco3482-jpg/Tobacco3482-jpg
/content/drive/MyDrive/Tobacco3482-jpg/Scientific
/content/drive/MyDrive/Tobacco3482-jpg/Memo
/content/drive/MyDrive/Tobacco3482-jpg/Note
/content/drive/MyDrive/Tobacco3482-jpg/Report
/content/drive/MyDrive/Tobacco3482-jpg/News
/content/drive/MyDrive/Tobacco3482-jpg/Email
/content/drive/MyDrive/Tobacco3482-jpg/Form
/content/drive/MyDrive/Tobacco3482-jpg/Resume
/content/drive/MyDrive/Tobacco3482-jpg/ADVE


In [3]:
def get_file_paths_and_labels(data_root):
     img_paths = [str(path) for path in data_root.glob('*/*.jpg')]
     labels = [p.split("/")[-2] for p in img_paths]
     return img_paths, labels

img_paths, labels = get_file_paths_and_labels(data_root)
# print(img_paths)
# print(labels)
print(len(img_paths))
print(len(labels))

3482
3482


In [4]:
df = pd.DataFrame(list(zip(img_paths, labels)),
               columns =['image_path', 'data_label'])
df.head()

,image_path,data_label
0,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,Letter
1,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,Letter
2,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,Letter
3,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,Letter
4,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,Letter


In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['data_label']= le.fit_transform(df['data_label'])
df.head()

,image_path,data_label
0,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,3
1,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,3
2,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,3
3,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,3
4,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,3


In [6]:
df['data_label'] = [str(i) for i in df['data_label']]

In [7]:
labels_dict = {}
for idx, this_path in enumerate(df['image_path']):
    labels_dict[this_path.split("/")[-2]] = df['data_label'][idx]
    
print(labels_dict)

{'Letter': '3', 'Scientific': '9', 'Memo': '4', 'Note': '6', 'Report': '7', 'News': '5', 'Email': '1', 'Form': '2', 'Resume': '8', 'ADVE': '0'}


In [8]:
balance=df['data_label'].value_counts()
print (balance)

4    620
1    599
3    567
2    431
7    265
9    261
0    230
6    201
5    188
8    120
Name: data_label, dtype: int64


In [9]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import random
random.seed(56)

In [12]:
batch_size=32
epochs = 1
fine_tune_epochs = 1
def scalar(x):
    return x/127.5-1

d = {
    "precision" : [],
     "recall" : [],
     "f1_score" : [],
     "accuracy" : [],
     "seed_value" : []
}
seed = []


for i in range(2):
    print('#Run: ', i+1)
    sd = random.randint(0, 100)
    seed.append(sd)

    train_df, dummy_df=train_test_split(df, test_size=0.3, shuffle=True,random_state=0)
    test_df, valid_df= train_test_split(dummy_df, test_size=0.5, shuffle=True,random_state=0)
    length=len(test_df)

    ######## Keras datagenerator #########
    trgen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=scalar)

    train_gen=trgen.flow_from_dataframe(train_df, x_col='image_path', y_col='data_label', target_size=(224,224), class_mode='categorical',batch_size=batch_size, shuffle=True, seed=sd)

    tvgen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=scalar)

    valid_gen=tvgen.flow_from_dataframe(valid_df, x_col='image_path', y_col='data_label', target_size=(224,224), class_mode='categorical',batch_size=batch_size, shuffle=False, seed=sd)

    # determine test generator batch size and steps to go through the test set exactly once for predictions
    test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=batch_size],reverse=True)[0]  

    test_steps=int(length/test_batch_size)

    test_gen=tvgen.flow_from_dataframe(test_df, x_col='image_path', y_col='data_label', target_size=(224,224), class_mode='categorical',batch_size=test_batch_size, shuffle=False, seed=sd)

    test_labels=test_gen.labels

    img_shape=(224,224,3)
    class_count = len(set(df['data_label']))
    neurons=512
    dropout=0.2
    lr= 0.001

    base_model=tf.keras.applications.MobileNetV2(include_top=False, input_shape=img_shape, pooling='max', weights='imagenet') 
    base_model.trainable=False

    inputs = keras.Input(shape=img_shape)
    x = base_model(inputs, training=False)

    # x=base_model.output
    # x=tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99 , epsilon=0.001 )(x)
    x =tf.keras.layers.Dense(512, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006), bias_regularizer=regularizers.l1(0.006) ,activation='relu', kernel_initializer= tf.keras.initializers.GlorotUniform(seed=sd))(x)
    outputs=tf.keras.layers.Dense(class_count, activation='softmax',kernel_initializer=tf.keras.initializers.GlorotUniform(seed=sd))(x)

    model=Model(inputs, outputs)

    model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])


    from keras.callbacks import ModelCheckpoint
    checkpoint = ModelCheckpoint("/model-{epoch:03d}-{accuracy:03f}-{val_accuracy:03f}.h5", verbose=1, monitor='val_loss',save_best_only=True, mode='auto') 

    history=model.fit(x=train_gen,  epochs= epochs, verbose=2, validation_data=valid_gen, callbacks=[checkpoint],
               validation_steps=None,  shuffle=False,  initial_epoch=0)
    

     ####### Fine-Tune ######
    base_model.trainable=True
    tune_epochs = fine_tune_epochs
    total_epochs= epochs + tune_epochs

    model.compile(Adamax(learning_rate=lr/10), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history_fine = model.fit(x=train_gen,  epochs=total_epochs, verbose=2, validation_data=valid_gen,
               validation_steps=None,  shuffle=False,  initial_epoch=epochs)
    

    y_pred = model.predict(test_gen)
    y_pred = np.argmax(y_pred, axis=1)

    y_true = test_gen.classes

    d["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d['accuracy'].append(float(accuracy_score(y_true, y_pred)))

#Run:  1
Found 2437 validated image filenames belonging to 10 classes.
Found 523 validated image filenames belonging to 10 classes.
Found 522 validated image filenames belonging to 10 classes.

Epoch 1: val_loss improved from inf to 9.12897, saving model to /model-001-0.498153-0.562141.h5
77/77 - 817s - loss: 11.4705 - accuracy: 0.4982 - val_loss: 9.1290 - val_accuracy: 0.5621 - 817s/epoch - 11s/step
Epoch 2/2
77/77 - 90s - loss: 8.3242 - accuracy: 0.5478 - val_loss: 7.5711 - val_accuracy: 0.5908 - 90s/epoch - 1s/step


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Run:  2
Found 2437 validated image filenames belonging to 10 classes.
Found 523 validated image filenames belonging to 10 classes.
Found 522 validated image filenames belonging to 10 classes.

Epoch 1: val_loss improved from inf to 9.47997, saving model to /model-001-0.473533-0.556405.h5
77/77 - 82s - loss: 12.0435 - accuracy: 0.4735 - val_loss: 9.4800 - val_accuracy: 0.5564 - 82s/epoch - 1s/step
Epoch 2/2
77/77 - 89s - loss: 8.5170 - accuracy: 0.5749 - val_loss: 7.8048 - val_accuracy: 0.5717 - 89s/epoch - 1s/step


In [ ]:
from google.colab import files
print(seed)
d['seed_value'] = seed
result = pd.DataFrame(d, index = ["run-"+str(i+1) for i in range(2)])
print(result)
result.to_csv("MobileNetV2.csv")
files.download('MobileNetV2_result.csv')